codesummary

In [1]:
#pip install pandas-gbq

In [1]:
import pandas as pd
import ast
from sklearn.metrics import mean_absolute_error
import datetime
import time
from pyactiveresource.util import to_query
from apscheduler.schedulers.blocking import BlockingScheduler
from pandas import json_normalize
from requests.exceptions import HTTPError
import os
import sys
from dateutil import parser
from sklearn.metrics import mean_squared_error
import itertools
from prophet import Prophet
import matplotlib.pyplot as plt
import numpy as np
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.model_selection import ParameterGrid
from math import sqrt
import logging
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from prophet.plot import plot_plotly, plot_components_plotly
from itertools import product
from tqdm import trange
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from prophet.plot import plot_plotly, plot_components_plotly
from itertools import product
from tqdm import tqdm, trange
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import xgboost as xgb
from sklearn.model_selection import train_test_split
import logging
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid
import warnings
import json
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_squared_error
from math import sqrt
import logging
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import logging
from prophet import Prophet
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import ParameterGrid, train_test_split
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import warnings
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_squared_error
from math import sqrt
import logging
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from prophet.plot import plot_plotly, plot_components_plotly
from itertools import product
from tqdm import tqdm, trange
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_squared_error
from math import sqrt
import logging
from tqdm import tqdm
import os
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from prophet.plot import plot_plotly, plot_components_plotly
from itertools import product
import pandas as pd
from datetime import datetime, timedelta
import pmdarima as pm
import calendar
from pandas.io import gbq
from sklearn.cross_decomposition import PLSRegression

C:\Users\erich\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#pip install gspread pandas oauth2client
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
mod = "Select * FROM panoply.ProphetX"

In [1]:
mod = "Select * FROM panoply.ProphetX"
df = gbq.read_gbq(mod,  project_id="xxx")

NameError: name 'gbq' is not defined

In [2]:
df

NameError: name 'df' is not defined

# Manual Loading Method
mod = '96a2855e-4da8-4559-acc0-c9093d268235_.csv'
df = pd.read_csv(mod, low_memory=False)
#df = ogdf

In [6]:
df['ds'] = pd.to_datetime(df['ds'])
df['launch_date'] = pd.to_datetime(df['launch_date']).dt.tz_localize(None)
ogdf = df


In [7]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

df['datetime'] = pd.to_datetime(df['ds'], utc=True)
df.set_index('datetime', inplace=True)
df = df[df.index >= '2022-09-01']


## Begin Core Arima


In [8]:
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year, month)[1])
    return datetime(year, month, day)

# Prediction start and end dates
today_date = datetime.now().date()
start_date = today_date + timedelta(days=1)
end_date = add_months(start_date, 4)

Monthly SARIMAX

In [9]:

class Model:
    def __init__(self):
        pass

    def fit(self, training_ds):
        pass

    def predict(self, predict_ds):
        pass 


class BaselineWeightedAverageModel(Model):
    def __init__(self):
        super().__init__()

    def predict(self, predict_ds, predict_date):
        skus = predict_ds['sku'].unique()
        predictions = {sku: 0 for sku in skus}

        dates = [predict_date + datetime.timedelta(days=-x) for x in [90, 60, 30]] + [predict_date]
        weights = [1/6, 1/3, 0.5]

        for weight, (start_date, end_date) in zip(weights, zip(dates[:-1], dates[1:])):
            date_slice = predict_ds[(predict_ds.index.date > start_date) &
                                    (predict_ds.index.date <= end_date)]
            print(weight, start_date, end_date)
            for sku in skus:
                relevant_orders = date_slice[(date_slice['sku'] == sku)]
                predictions[sku] += weight * relevant_orders['quantity'].sum() 
            
        return predictions

def subtract_one_month(date):
    m, y = (date.month-1) % 12, date.year + ((date.month)-2) // 12
    if not m: m = 12
    d = min(date.day, calendar.monthrange(y, m)[1])
    return date.replace(day=d,month=m, year=y)

def slice_quantities_by_column(ds, end_date, column, max_lags=None, slice_by_month=False):
    sliced_quantity_by_sku = {sku: [] for sku in ds[column].unique()}
    n_total = len(ds[ds.index.date <= end_date])
    n_sliced, n_slices = 0, 0

    cur_slice_end_date = end_date
    while n_sliced < n_total:
        if not slice_by_month:
            cur_slice_start_date = cur_slice_end_date + timedelta(days=-30)
        else:
            cur_slice_start_date = subtract_one_month(cur_slice_end_date)
            
        date_slice = ds[(ds.index.date > cur_slice_start_date) &
                                (ds.index.date <= cur_slice_end_date)]
        quantity_by_sku = date_slice.groupby(column)['quantity'].sum().to_dict()
        for sku in sliced_quantity_by_sku:
            sliced_quantity_by_sku[sku].append(quantity_by_sku.get(sku, 0))

        n_sliced += len(date_slice)
        n_slices += 1
        print(f'{n_slices} ({cur_slice_start_date} - {cur_slice_end_date}): {n_sliced}/{n_total}')
        if max_lags and max_lags <= n_slices:
            break
        cur_slice_end_date = cur_slice_start_date

    for k in sliced_quantity_by_sku:
        sliced_quantity_by_sku[k] = sliced_quantity_by_sku[k][::-1]

    return sliced_quantity_by_sku    

class PerSkuARIMAModel(Model):
    def __init__(self, target_period_months, seasonal=False, slice_by_month=False):
        super().__init__()

        self._target_period_months = target_period_months
        self._predictor_by_sku = {}
        self._aic_by_sku = {}  # Store AIC values by SKU
        self._seasonal = seasonal
        self._seasonal_m = 12 if seasonal else 0
        self._slice_by_month = slice_by_month

    def _slice_quantities(self, predict_ds, predict_date, max_lags=None):
        return slice_quantities_by_column(predict_ds, predict_date, column='sku', max_lags=max_lags, slice_by_month=self._slice_by_month)

    def _fit(self, sliced_quantities):
        self._predictor_by_sku = {}

        for sku, series in sliced_quantities.items():
            model = \
                pm.auto_arima(series, error_action='ignore', trace=True,
                suppress_warnings=True, maxiter=5, seasonal=self._seasonal, 
                m=self._seasonal_m, D=0)
            
            self._predictor_by_sku[sku] = model
            self._aic_by_sku[sku] = model.aic()  # Store AIC value

    def predict(self, predict_ds, predict_date):
        sliced_quantities = self._slice_quantities(predict_ds, predict_date)

        self._fit(sliced_quantities)
        
        skus = predict_ds['sku'].unique()
        predictions = {sku: 0 for sku in skus}

        for sku in skus:
            model = self._predictor_by_sku[sku]
            preds = model.predict(n_periods=self._target_period_months+1)
                
            predictions[sku] = preds[:self._target_period_months].sum()
            
        return predictions

class PLSModel(Model):
    def __init__(self, target_period_months, lag_period_months, slice_by_month=False, n_components=1):
        super().__init__()

        self._target_period_months = target_period_months
        self._lag_period_months = lag_period_months
        self._slice_by_month = slice_by_month
        self._n_components = n_components
        self._aic = None  # Store AIC value

    def _slice_quantities(self, predict_ds, predict_date, max_lags=None):
        return slice_quantities_by_column(predict_ds, predict_date, column='sku', max_lags=max_lags, slice_by_month=self._slice_by_month)

    def predict(self, predict_ds, predict_date):
        sliced_quantities = self._slice_quantities(predict_ds, predict_date)

        series_length = len(next(iter(sliced_quantities.values())))
        n = series_length - self._lag_period_months - self._target_period_months
        m = len(sliced_quantities)
        
        mat_predictors = np.zeros((n, m))
        mat_targets = np.zeros((n, m))
        oos_predictors = np.zeros((1, m))

        for idx_sku, (sku, sales_series) in enumerate(sliced_quantities.items()):
            for i in range(n):
                mat_targets[i, idx_sku] = sum(sales_series[i+self._lag_period_months:i+self._lag_period_months+self._target_period_months-1])
                mat_predictors[i, idx_sku] = sum(sales_series[i:i+self._lag_period_months])

            oos_predictors[0, idx_sku] = sum(sales_series[-self._lag_period_months:])                
        
        pls = PLSRegression(n_components=self._n_components)
        pls.fit(mat_predictors, mat_targets)

        pls_oos_predictions = {}
        raw_preds = pls.predict(oos_predictors)
        for idx_sku, sku in enumerate(sliced_quantities):
            pls_oos_predictions[sku] = max(float(raw_preds[0, idx_sku]), 0)
            
        return pls_oos_predictions

class CompositePerSKUArimaPLSModel(Model):
    def __init__(self, target_period_months, slice_by_month=False, n_components=1):
        super().__init__()

        self._target_period_months = target_period_months
        self._slice_by_month = slice_by_month

    def predict(self, predict_ds, predict_date):
        arima = PerSkuARIMAModel(target_period_months=self._target_period_months, slice_by_month=self._slice_by_month)
        pls = PLSModel(target_period_months=self._target_period_months, lag_period_months=self._target_period_months, slice_by_month=self._slice_by_month)

        preds_arima = arima.predict(predict_ds, predict_date)
        preds_pls = pls.predict(predict_ds, predict_date)

        return {k: 0.5 * (v + preds_pls[k]) for k, v in preds_arima.items()}       

In [10]:
# Create the composite model
fwam = CompositePerSKUArimaPLSModel(target_period_months=4)
preds = fwam.predict(df, predict_date=start_date)

# Create predictions_df DataFrame
predictions_df = pd.DataFrame(list(preds.items()), columns=['SKU', 'Predicted Sales'])

predictions_df['Prediction Date'] = datetime.now().date()

today_date = datetime.now().date()
start_date = today_date # + timedelta(days=1)
fwam = CompositePerSKUArimaPLSModel(target_period_months=4)
preds = fwam.predict(df, predict_date=start_date)

1 (2024-05-13 - 2024-06-12): 0/112193
2 (2024-04-13 - 2024-05-13): 0/112193
3 (2024-03-14 - 2024-04-13): 0/112193
4 (2024-02-13 - 2024-03-14): 0/112193
5 (2024-01-14 - 2024-02-13): 0/112193
6 (2023-12-15 - 2024-01-14): 0/112193
7 (2023-11-15 - 2023-12-15): 0/112193
8 (2023-10-16 - 2023-11-15): 703/112193
9 (2023-09-16 - 2023-10-16): 9703/112193
10 (2023-08-17 - 2023-09-16): 18906/112193
11 (2023-07-18 - 2023-08-17): 29356/112193
12 (2023-06-18 - 2023-07-18): 37785/112193
13 (2023-05-19 - 2023-06-18): 46538/112193
14 (2023-04-19 - 2023-05-19): 55942/112193
15 (2023-03-20 - 2023-04-19): 60552/112193
16 (2023-02-18 - 2023-03-20): 66659/112193
17 (2023-01-19 - 2023-02-18): 72261/112193
18 (2022-12-20 - 2023-01-19): 77010/112193
19 (2022-11-20 - 2022-12-20): 86982/112193
20 (2022-10-21 - 2022-11-20): 95965/112193
21 (2022-09-21 - 2022-10-21): 105218/112193
22 (2022-08-22 - 2022-09-21): 112193/112193
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=195.797

 ARIMA(2,0,1)(0,0,0)[0]             : AIC=195.388, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=192.033, Time=0.02 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0]          
Total fit time: 0.093 seconds
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=227.683, Time=0.02 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=250.539, Time=0.00 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=222.533, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=236.539, Time=0.01 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=224.314, Time=0.01 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=224.678, Time=0.01 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=226.740, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=223.880, Time=0.01 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0]          
Total fit time: 0.103 seconds
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=190.581, Time=0.02 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=205

 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=228.445, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=228.374, Time=0.01 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=225.463, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=230.257, Time=0.02 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.089 seconds
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=135.845, Time=0.02 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=135.906, Time=0.00 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=135.867, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=131.681, Time=0.01 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=134.124, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=133.084, Time=0.01 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=131.935, Time=0.01 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=133.858, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=131.673, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0]            

 ARIMA(1,0,1)(0,0,0)[0]             : AIC=197.149, Time=0.01 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=189.241, Time=0.01 sec
 ARIMA(0,0,3)(0,0,0)[0]             : AIC=202.414, Time=0.01 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=198.855, Time=0.01 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=190.930, Time=0.02 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=188.823, Time=0.02 sec

Best model:  ARIMA(1,0,2)(0,0,0)[0]          
Total fit time: 0.151 seconds
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=173.079, Time=0.02 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=169.170, Time=0.00 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=170.638, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=170.170, Time=0.01 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=167.195, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=172.195, Time=0.01 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.068 seconds
Performing stepwise sea

 ARIMA(1,0,2)(0,0,0)[0]             : AIC=200.609, Time=0.03 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=206.808, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=198.601, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=197.253, Time=0.01 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=211.459, Time=0.00 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=198.627, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=201.708, Time=0.01 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=200.413, Time=0.01 sec

Best model:  ARIMA(0,0,1)(0,0,0)[0] intercept
Total fit time: 0.212 seconds
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=148.312, Time=0.02 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=147.617, Time=0.00 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=144.661, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=142.309, Time=0.01 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=144.309, Time=0.01 sec
 ARIMA(0,0,2)(0,0,0)[0]            

...

 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-2.593, Time=0.02 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0]          
Total fit time: 0.132 seconds
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=5874.168, Time=0.05 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=26.929, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=5868.168, Time=0.03 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=5868.168, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=5870.168, Time=0.03 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=27.905, Time=0.02 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0]          
Total fit time: 0.170 seconds
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=4.430, Time=0.01 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-3.570, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=-1.570, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=-1.570, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=0.430, 

 ARIMA(0,0,2)(0,0,0)[0]             : AIC=3.267, Time=0.01 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=5.346, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=3.419, Time=0.01 sec

Best model:  ARIMA(0,0,1)(0,0,0)[0]          
Total fit time: 0.104 seconds
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=19.680, Time=0.02 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=11.680, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=13.680, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=13.680, Time=0.01 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=15.680, Time=0.01 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=11.583, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=13.349, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=13.269, Time=0.01 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=15.309, Time=0.02 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0] intercept
Total fit time: 0.111 seconds
Performing stepwise search to minimize

 ARIMA(1,0,0)(0,0,0)[0]             : AIC=69.246, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=69.246, Time=0.01 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=71.246, Time=0.01 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=68.222, Time=0.00 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0]          
Total fit time: 0.107 seconds
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=75.246, Time=0.02 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=67.246, Time=0.00 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=69.246, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=69.246, Time=0.01 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=71.246, Time=0.01 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=68.222, Time=0.00 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0]          
Total fit time: 0.053 seconds
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=34.929, Time=0.02 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=26.929, Time=0

 ARIMA(2,0,2)(0,0,0)[0]             : AIC=5874.168, Time=0.04 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=26.929, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=5868.168, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=5868.168, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=5870.168, Time=0.02 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=27.905, Time=0.01 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0]          
Total fit time: 0.125 seconds
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=4.430, Time=0.01 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-3.570, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=-1.570, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=-1.570, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=0.430, Time=0.02 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-2.593, Time=0.01 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0]          
Total fit time: 0.082 seconds
Performing stepwise search to

 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=106.743, Time=0.01 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0]          
Total fit time: 0.051 seconds
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=1391.429, Time=0.04 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-3.570, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=-1.570, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=1385.429, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=1387.429, Time=0.02 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-2.593, Time=0.01 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0]          
Total fit time: 0.114 seconds
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=4.430, Time=0.01 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-3.570, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=-1.570, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=-1.570, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=0.430, T

In [15]:
def eval_predictions(df_eval, prediction_by_sku):
    df_eval_by_sku = df_eval.groupby('sku')
    actual_quanity_by_sku = df_eval_by_sku['quantity'].sum().to_dict()
    average_price_by_sku = (df_eval_by_sku['price'].sum() / df_eval_by_sku['quantity'].sum()).to_dict()

    total_price_val = 0
    total_quantity_val = 0
    total_quantity_delta = 0
    total_price_delta = 0

    for sku, pred in prediction_by_sku.items():
        if not sku in actual_quanity_by_sku:
            continue

         
        quantity_delta = actual_quanity_by_sku[sku] - pred 
        price_delta = quantity_delta * average_price_by_sku[sku]
        
        total_quantity_val += actual_quanity_by_sku[sku]
        total_price_val += actual_quanity_by_sku[sku] * average_price_by_sku[sku]
        total_quantity_delta += abs(quantity_delta)
        total_price_delta += abs(price_delta)
    
    return {
        'total_quantity_delta': total_quantity_delta,
        'total_price_delta': total_price_delta,
        'rel_quantity_delta': total_quantity_delta / total_quantity_val,
        'rel_price_delta': total_price_delta / total_price_val
    }

In [16]:
preds

{'GBB2002YA1': 50.07277825002312,
 'GBR1980YM': 190.0588932357332,
 'GBR1980YL': 196.31534238390046,
 'GBK1810Y2': 83.97927279658462,
 'GBR1970MS': 110.46431692634157,
 'GBR1970MXL': 134.79667932066414,
 'GBK1913A0': 142.4018558089098,
 'GBR1970MM': 168.31635012557376,
 'GBT2170ML': 13.143144379092229,
 'GBB2002YY4': 132.9057931351699,
 'GBRX1962ML': 14.803802814252334,
 'GBR1970M2XL': 91.82534735463447,
 'GBB1903A3': 51.83063870697246,
 'GBB2007YY5': 5.723397847951614,
 'GBB2007YY4': 1.1843216305758935,
 'GBB2002YY2': 48.48870327065139,
 'GBB2005YY5': 20.86114645157511,
 'GBB2007YY6': 6.05834910086778,
 'GBK1809Y2': 129.44519705905273,
 'GBB2002YY1': 13.124500207633345,
 'GBK1913A1': 222.83415279260515,
 'GBK1809Y4': 145.08085291813325,
 'GBK1810Y0': 8.668577217061298,
 'GBL1990WM': 2.950037888073429,
 'GBK1913A3': 278.3683346646501,
 'GBB1902A3': 70.30466083518475,
 'GBB2004YY6': 19.739649329874695,
 'GBB1904A2': 28.201362919959408,
 'GBB2002YY5': 96.18407655390081,
 'GBB1904A3': 26.

In [18]:
predictions_df = pd.DataFrame(list(preds.items()), columns=['SKU', 'Predicted Sales'])
forecast_period_days = 120

# Add 'Prediction Date' which is today
predictions_df['Prediction Date'] = datetime.now().date()

# Assume 'Start of Forecast Date' is the day after the prediction date
predictions_df['Start of Forecast Date'] = predictions_df['Prediction Date'] + timedelta(days=1)

predictions_df['Forecast End Date'] = predictions_df['Start of Forecast Date'] + timedelta(days=forecast_period_days - 1)

# Calculate 'Average Daily Sales Rate'
predictions_df['Average Daily Sales Rate'] = predictions_df['Predicted Sales'] / forecast_period_days
predictions_df[f'{forecast_period_days}X'] = forecast_period_days * predictions_df['Average Daily Sales Rate']

# View the DataFrame
predictions_df

,SKU,Predicted Sales,Prediction Date,Start of Forecast Date,Forecast End Date,Average Daily Sales Rate,120X
0,GBB2002YA1,50.072778,2024-06-11,2024-06-12,2024-10-09,0.417273,50.072778
1,GBR1980YM,190.058893,2024-06-11,2024-06-12,2024-10-09,1.583824,190.058893
2,GBR1980YL,196.315342,2024-06-11,2024-06-12,2024-10-09,1.635961,196.315342
3,GBK1810Y2,83.979273,2024-06-11,2024-06-12,2024-10-09,0.699827,83.979273
4,GBR1970MS,110.464317,2024-06-11,2024-06-12,2024-10-09,0.920536,110.464317
...,...,...,...,...,...,...,...
2198,GBT2120WXS,0.000000,2024-06-11,2024-06-12,2024-10-09,0.000000,0.000000
2199,GBT2089WL,0.000000,2024-06-11,2024-06-12,2024-10-09,0.000000,0.000000
2200,GBCH2003GRY - WS,0.425259,2024-06-11,2024-06-12,2024-10-09,0.003544,0.425259
2201,GBSB1903WXL,0.106017,2024-06-11,2024-06-12,2024-10-09,0.000883,0.106017


In [19]:
predictions_df.to_csv("predictions.csv")

## End Sarimax Core | Begin Supplementary Files

Inventory

In [16]:
df = ogdf
df.dropna(subset=['sku'], inplace=True)

pd.to_datetime(df['launch_date'], utc=True)

end_date = df['ds'].max()
start_date = end_date - pd.DateOffset(months=36)
inventory = df[['sku', 'inventory_quantity']]
inventory_unique = inventory.drop_duplicates(subset=['sku'])

inventory_unique.to_csv('inventory.csv', index=False)

Drop inventory; trim useless columns and impute where necessary

In [17]:
df

,OrderType,order_name,created_at,age,school_info,sku,product_color,inventory_quantity,price_per_unit,vendor,quantity,price,cost,variant_title,line_item_name,DATE,ds,launch_date,product_age_days,category,sub_category,product_type,published_at,status
datetime,,,,,,,,,,,,,,,,,,,,,,,,
2024-06-06 00:00:00+00:00,B2C,GBW-175087,2024-06-06 11:55:10+00:00,Adult,None,GBT2228MXL,Royal,3,29.990000000,Inkwright,1,29.990000000,7.310000000,XL,Legacy Mens Tee - Royal - XL,2024-06-06 00:00:00+00:00,2024-06-06,2024-03-12,44,Casual,None,Shirt,2024-04-23 15:58:43+00:00,L
2024-06-06 00:00:00+00:00,B2C,GBW-175087,2024-06-06 11:55:10+00:00,Adult,None,GBT2204MXL,Black,35,34.990000000,Inkwright,1,34.990000000,11.010000000,XL,RS Mens L/S Tee - Black - XL,2024-06-06 00:00:00+00:00,2024-06-06,2023-12-15,128,Casual,None,Shirt,2024-01-30 17:00:56+00:00,R
2024-06-06 00:00:00+00:00,B2C,GBW-175087,2024-06-06 11:55:10+00:00,Adult,None,GBSTCK04,Red,36,8.000000000,Specialized Stickers,1,8.000000000,3.120000000,OS,GB Assorted Matte Sticker Sheet - Red - OS,2024-06-06 00:00:00+00:00,2024-06-06,2022-07-06,665,Accessory,None,Sticker,2022-08-11 19:45:33+00:00,R
2024-06-06 00:00:00+00:00,B2C,GBW-175087,2024-06-06 11:55:10+00:00,All,None,GBSTCK01,RED,1019,1.500000000,GB Edition,1,1.500000000,0.300000000,"3""","GB Round Sticker 3"" x 3"" - RED - 3""",2024-06-06 00:00:00+00:00,2024-06-06,2018-05-01,2228,Accessory,None,Sticker,2018-05-01 22:06:22+00:00,A
2024-06-06 00:00:00+00:00,B2C,GBW-175087,2024-06-06 11:55:10+00:00,Youth,None,GBR1985YM,Grey,115,42.990000000,Novo Estrada Limited,2,42.990000000,9.850000000,YM,GB Edition S/S Youth Ranked Rashguard - Grey - YM,2024-06-06 00:00:00+00:00,2024-06-06,2024-03-15,2,Training,Core,Rash Guard,2024-06-04 16:00:10+00:00,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-01 00:00:00+00:00,B2C,GBW-106019,2021-08-01 02:44:57+00:00,Adult,Mckinney,GBR1936L,White,0,44.990000000,Novo Estrada Limited,1,44.990000000,10.500000000,L,Mens Ranked Rashguard V2 - White - L,2021-08-01 00:00:00+00:00,2021-08-01,2021-03-18,1176,Training,Core,Rash Guard,NaT,D
2021-08-01 00:00:00+00:00,B2C,GBW-106018,2021-08-01 00:34:07+00:00,Adult,Mckinney,GBB2007BR,Black/Red,0,64.990000000,Quanzhou Pengfei Bags,1,64.990000000,15.480000000,ALL,GB3 Tech Backpack - Black/Red - ALL,2021-08-01 00:00:00+00:00,2021-08-01,2020-12-23,1261,Accessory,None,Backpack,NaT,D
2021-08-01 00:00:00+00:00,B2C,GBW-106018,2021-08-01 00:34:07+00:00,Adult,Mckinney,GBHW1041,Black Camo,0,28.000000000,LogoMart,1,28.000000000,10.660000000,OS,Stacked Snapback - Black Camo - OS,2021-08-01 00:00:00+00:00,2021-08-01,2021-02-04,1218,Accessory,None,Headwear,NaT,D


def fill_category_from_nearest(df):
    # Iterate over each row in the DataFrame
    for idx, row in df.iterrows():
        # Check if 'category' is NaN
        if pd.isnull(row['category']):
            # Find rows with the same 'product_type' and a non-NaN 'category'
            same_type_non_nan = df[(df['product_type'] == row['product_type']) & (~df['category'].isna())]
            if not same_type_non_nan.empty:
                # Pick the first available 'category' from the filtered DataFrame
                df.at[idx, 'category'] = same_type_non_nan.iloc[0]['category']

fill_category_from_nearest(df)

df

In [18]:
column_names = df.columns.tolist()

# Print the column names
print(column_names)

['OrderType', 'order_name', 'created_at', 'age', 'school_info', 'sku', 'product_color', 'inventory_quantity', 'price_per_unit', 'vendor', 'quantity', 'price', 'cost', 'variant_title', 'line_item_name', 'DATE', 'ds', 'launch_date', 'product_age_days', 'category', 'sub_category', 'product_type', 'published_at', 'status']


In [19]:
keep = ['ds','sku','line_item_name','quantity','product_type','product_color','cost','price_per_unit', 'category', 'sub_category',  'age','launch_date', 'status','variant_title','vendor']
df = df[keep]
df['launch_date'] = pd.to_datetime(df['launch_date'], errors='coerce')

df = df[~df['sku'].str.contains('TEST', na=False)]
df['age'].fillna('All', inplace=True)

df['category'] = df.groupby('product_type')['category'].transform(lambda x: x.ffill().bfill())

# Impute all 'product_color' with N/A
df['product_color'].fillna('N/A', inplace=True)

# Impute all 'sub_category' with Non-Core
df['sub_category'].fillna('Non-Core', inplace=True)

# Impute all 'published_at' with '1899-01-01' as date value
df['launch_date'].fillna(pd.to_datetime('1899-01-01'), inplace=True)

# Impute all 'vendor' with ?
df['vendor'].fillna('?', inplace=True)

# Impute all 'variant_title' with OS
df['variant_title'].fillna('OS', inplace=True)

# Impute all 'status' with ?
df['status'].fillna('?', inplace=True)
df

C:\Users\erich\AppData\Local\Temp\ipykernel_21008\2010636955.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['launch_date'] = pd.to_datetime(df['launch_date'], errors='coerce')
C:\Users\erich\AppData\Local\Temp\ipykernel_21008\2010636955.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original obj

,ds,sku,line_item_name,quantity,product_type,product_color,cost,price_per_unit,category,sub_category,age,launch_date,status,variant_title,vendor
datetime,,,,,,,,,,,,,,,
2024-06-06 00:00:00+00:00,2024-06-06,GBT2228MXL,Legacy Mens Tee - Royal - XL,1,Shirt,Royal,7.310000000,29.990000000,Casual,Non-Core,Adult,2024-03-12,L,XL,Inkwright
2024-06-06 00:00:00+00:00,2024-06-06,GBT2204MXL,RS Mens L/S Tee - Black - XL,1,Shirt,Black,11.010000000,34.990000000,Casual,Non-Core,Adult,2023-12-15,R,XL,Inkwright
2024-06-06 00:00:00+00:00,2024-06-06,GBSTCK04,GB Assorted Matte Sticker Sheet - Red - OS,1,Sticker,Red,3.120000000,8.000000000,Accessory,Non-Core,Adult,2022-07-06,R,OS,Specialized Stickers
2024-06-06 00:00:00+00:00,2024-06-06,GBSTCK01,"GB Round Sticker 3"" x 3"" - RED - 3""",1,Sticker,RED,0.300000000,1.500000000,Accessory,Non-Core,All,2018-05-01,A,"3""",GB Edition
2024-06-06 00:00:00+00:00,2024-06-06,GBR1985YM,GB Edition S/S Youth Ranked Rashguard - Grey - YM,2,Rash Guard,Grey,9.850000000,42.990000000,Training,Core,Youth,2024-03-15,N,YM,Novo Estrada Limited
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-01 00:00:00+00:00,2021-08-01,GBR1936L,Mens Ranked Rashguard V2 - White - L,1,Rash Guard,White,10.500000000,44.990000000,Training,Core,Adult,2021-03-18,D,L,Novo Estrada Limited
2021-08-01 00:00:00+00:00,2021-08-01,GBB2007BR,GB3 Tech Backpack - Black/Red - ALL,1,Backpack,Black/Red,15.480000000,64.990000000,Accessory,Non-Core,Adult,2020-12-23,D,ALL,Quanzhou Pengfei Bags
2021-08-01 00:00:00+00:00,2021-08-01,GBHW1041,Stacked Snapback - Black Camo - OS,1,Headwear,Black Camo,10.660000000,28.000000000,Accessory,Non-Core,Adult,2021-02-04,D,OS,LogoMart


In [20]:
#df['launch_date'].fillna('NA', inplace=True)
#df['launch_date'].replace('NA', pd.Timestamp('1899-01-01'), inplace=True)

nans_per_column = df.isna().sum()
# Print each column name along with the number of NaNs it contains
for column, nan_count in nans_per_column.items():
    print(f"{column}: {nan_count} NaNs")

ds: 0 NaNs
sku: 0 NaNs
line_item_name: 0 NaNs
quantity: 0 NaNs
product_type: 379 NaNs
product_color: 0 NaNs
cost: 657 NaNs
price_per_unit: 381 NaNs
category: 513 NaNs
sub_category: 0 NaNs
age: 0 NaNs
launch_date: 0 NaNs
status: 0 NaNs
variant_title: 0 NaNs
vendor: 0 NaNs


In [21]:
nan_product_type_df = df[df['product_type'].isna()]
nan_product_type_df

,ds,sku,line_item_name,quantity,product_type,product_color,cost,price_per_unit,category,sub_category,age,launch_date,status,variant_title,vendor
datetime,,,,,,,,,,,,,,,
2023-03-12 00:00:00+00:00,2023-03-12,GBS1928UBLK28,Barra Performance V2 Short by Adidas® - Black ...,1,None,N/A,None,None,NaN,Non-Core,All,1899-01-01,?,28,Adidas
2023-03-10 00:00:00+00:00,2023-03-10,GBR1932UBLKL,Barra Performance V2 Rashguard L/S by Adidas® ...,2,None,N/A,None,None,NaN,Non-Core,All,1899-01-01,?,L,Adidas
2022-01-21 00:00:00+00:00,2022-01-21,GBKX1925F0,GB Pink Ribbon Kimono by Adidas® - White - F0,2,None,N/A,None,None,NaN,Non-Core,All,1899-01-01,?,F0,Double D
2022-01-18 00:00:00+00:00,2022-01-18,GBKX1925F0,GB Pink Ribbon Kimono by Adidas® - White - F0,1,None,N/A,None,None,NaN,Non-Core,All,1899-01-01,?,F0,Double D
2022-01-18 00:00:00+00:00,2022-01-18,GBKX1925F4,GB Pink Ribbon Kimono by Adidas® - White - F4,1,None,N/A,None,None,NaN,Non-Core,All,1899-01-01,?,F4,Double D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-03 00:00:00+00:00,2021-08-03,GBR1931UBLKXS,Barra Performance V2 Rashguard S/S by Adidas® ...,1,None,N/A,None,None,NaN,Non-Core,All,1899-01-01,?,XS,Adidas
2021-08-03 00:00:00+00:00,2021-08-03,GBS1928UBLK30,Barra Performance V2 Short by Adidas® - Black ...,1,None,N/A,None,None,NaN,Non-Core,All,1899-01-01,?,30,Adidas
2021-08-03 00:00:00+00:00,2021-08-03,GBS1928UBLK30,Barra Performance V2 Short by Adidas® - Black ...,1,None,N/A,None,None,NaN,Non-Core,All,1899-01-01,?,30,Adidas


In [22]:
columns_with_nans = ['sku', 'product_type', 'product_color', 'cost', 'price_per_unit', 'category', 
                     'age', 'status', 'variant_title', 'vendor']

# Filter rows where any of the specified columns have NaN values
rows_with_nans = df[df[columns_with_nans].isnull().any(axis=1)]
rows_with_nans

,ds,sku,line_item_name,quantity,product_type,product_color,cost,price_per_unit,category,sub_category,age,launch_date,status,variant_title,vendor
datetime,,,,,,,,,,,,,,,
2024-06-04 00:00:00+00:00,2024-06-04,GBGC25,Gift Card - $25,1,Gift Cards,N/A,None,25.000000000,Other,Non-Core,All,2018-11-14,D,$25,GB Edition
2024-05-22 00:00:00+00:00,2024-05-22,GBGC25,Gift Card - $25,1,Gift Cards,N/A,None,25.000000000,Other,Non-Core,All,2018-11-14,D,$25,GB Edition
2024-05-20 00:00:00+00:00,2024-05-20,GBGC100,Gift Card - $100,1,Gift Cards,N/A,None,100.000000000,Other,Non-Core,All,2018-11-14,D,$100,GB Edition
2024-05-18 00:00:00+00:00,2024-05-18,GBGC100,Gift Card - $100,1,Gift Cards,N/A,None,100.000000000,Other,Non-Core,All,2018-11-14,D,$100,GB Edition
2024-05-14 00:00:00+00:00,2024-05-14,GBGC200,Gift Card - $200,1,Gift Cards,N/A,None,200.000000000,Other,Non-Core,All,2018-11-14,D,$200,GB Edition
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-03 00:00:00+00:00,2021-08-03,GBR1931UBLKXS,Barra Performance V2 Rashguard S/S by Adidas® ...,1,None,N/A,None,None,NaN,Non-Core,All,1899-01-01,?,XS,Adidas
2021-08-03 00:00:00+00:00,2021-08-03,GBS1928UBLK30,Barra Performance V2 Short by Adidas® - Black ...,1,None,N/A,None,None,NaN,Non-Core,All,1899-01-01,?,30,Adidas
2021-08-03 00:00:00+00:00,2021-08-03,GBS1928UBLK30,Barra Performance V2 Short by Adidas® - Black ...,1,None,N/A,None,None,NaN,Non-Core,All,1899-01-01,?,30,Adidas


In [23]:
df

,ds,sku,line_item_name,quantity,product_type,product_color,cost,price_per_unit,category,sub_category,age,launch_date,status,variant_title,vendor
datetime,,,,,,,,,,,,,,,
2024-06-06 00:00:00+00:00,2024-06-06,GBT2228MXL,Legacy Mens Tee - Royal - XL,1,Shirt,Royal,7.310000000,29.990000000,Casual,Non-Core,Adult,2024-03-12,L,XL,Inkwright
2024-06-06 00:00:00+00:00,2024-06-06,GBT2204MXL,RS Mens L/S Tee - Black - XL,1,Shirt,Black,11.010000000,34.990000000,Casual,Non-Core,Adult,2023-12-15,R,XL,Inkwright
2024-06-06 00:00:00+00:00,2024-06-06,GBSTCK04,GB Assorted Matte Sticker Sheet - Red - OS,1,Sticker,Red,3.120000000,8.000000000,Accessory,Non-Core,Adult,2022-07-06,R,OS,Specialized Stickers
2024-06-06 00:00:00+00:00,2024-06-06,GBSTCK01,"GB Round Sticker 3"" x 3"" - RED - 3""",1,Sticker,RED,0.300000000,1.500000000,Accessory,Non-Core,All,2018-05-01,A,"3""",GB Edition
2024-06-06 00:00:00+00:00,2024-06-06,GBR1985YM,GB Edition S/S Youth Ranked Rashguard - Grey - YM,2,Rash Guard,Grey,9.850000000,42.990000000,Training,Core,Youth,2024-03-15,N,YM,Novo Estrada Limited
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-01 00:00:00+00:00,2021-08-01,GBR1936L,Mens Ranked Rashguard V2 - White - L,1,Rash Guard,White,10.500000000,44.990000000,Training,Core,Adult,2021-03-18,D,L,Novo Estrada Limited
2021-08-01 00:00:00+00:00,2021-08-01,GBB2007BR,GB3 Tech Backpack - Black/Red - ALL,1,Backpack,Black/Red,15.480000000,64.990000000,Accessory,Non-Core,Adult,2020-12-23,D,ALL,Quanzhou Pengfei Bags
2021-08-01 00:00:00+00:00,2021-08-01,GBHW1041,Stacked Snapback - Black Camo - OS,1,Headwear,Black Camo,10.660000000,28.000000000,Accessory,Non-Core,Adult,2021-02-04,D,OS,LogoMart


In [24]:
def fill_category_from_nearest(df):
    # Iterate over each row in the DataFrame
    for idx, row in df.iterrows():
        # Check if 'category' is NaN
        if pd.isnull(row['category']):
            # Find rows with the same 'product_type' and a non-NaN 'category'
            same_type_non_nan = df[(df['product_type'] == row['product_type']) & (~df['category'].isna())]
            if not same_type_non_nan.empty:
                # Pick the first available 'category' from the filtered DataFrame
                df.at[idx, 'category'] = same_type_non_nan.iloc[0]['category']

fill_category_from_nearest(df)

In [25]:
pd.to_datetime(df['launch_date'])

#df.set_index('ds', inplace=True)


# Filtering data
df = df[df['ds'] >= '2022-09-01']

# Remove SKUs containing 'TEST' and fill missing values


# Apply function to fill missing 'category'
fill_category_from_nearest(df)

# Remove duplicate rows based on 'category' and 'product_type' to get unique combinations
unique_combinations = df.drop_duplicates(subset=['category', 'product_type'])

# Additional calculations
unique_skus = df['product_type'].unique()
date_range_days = (df['ds'].max() - df['ds'].min()).days + 1  # +1 to include both start and end dates
total_rows = df.shape[0]
unique_skus_count = df['sku'].nunique()
max_possible_rows = unique_skus_count * date_range_days
total_sales = df['quantity'].sum()

# Print or analyze DataFrame
print("Unique Combinations:", unique_combinations)
print("Total NaN count:", df.isna().sum().sum())
print("Total Sales:", total_sales)

# Display DataFrame (optional)
df


Unique Combinations:                                   ds                sku  \
datetime                                                  
2024-06-06 00:00:00+00:00 2024-06-06         GBT2228MXL   
2024-06-06 00:00:00+00:00 2024-06-06           GBSTCK04   
2024-06-06 00:00:00+00:00 2024-06-06          GBR1985YM   
2024-06-06 00:00:00+00:00 2024-06-06            GBKEY05   
2024-06-06 00:00:00+00:00 2024-06-06          GBS2060YM   
2024-06-06 00:00:00+00:00 2024-06-06          GBK1967A3   
2024-06-06 00:00:00+00:00 2024-06-06          GBB1902A3   
2024-06-06 00:00:00+00:00 2024-06-06          GB3038BLK   
2024-06-06 00:00:00+00:00 2024-06-06          GBK1809Y5   
2024-06-06 00:00:00+00:00 2024-06-06          GBP1935MM   
2024-06-06 00:00:00+00:00 2024-06-06          GBR1957YM   
2024-06-06 00:00:00+00:00 2024-06-06         GBT2217MXL   
2024-06-06 00:00:00+00:00 2024-06-06         GBB2002YY5   
2024-06-06 00:00:00+00:00 2024-06-06         GBX3032GRY   
2024-06-06 00:00:00+00:00 2024-06-0

,ds,sku,line_item_name,quantity,product_type,product_color,cost,price_per_unit,category,sub_category,age,launch_date,status,variant_title,vendor
datetime,,,,,,,,,,,,,,,
2024-06-06 00:00:00+00:00,2024-06-06,GBT2228MXL,Legacy Mens Tee - Royal - XL,1,Shirt,Royal,7.310000000,29.990000000,Casual,Non-Core,Adult,2024-03-12,L,XL,Inkwright
2024-06-06 00:00:00+00:00,2024-06-06,GBT2204MXL,RS Mens L/S Tee - Black - XL,1,Shirt,Black,11.010000000,34.990000000,Casual,Non-Core,Adult,2023-12-15,R,XL,Inkwright
2024-06-06 00:00:00+00:00,2024-06-06,GBSTCK04,GB Assorted Matte Sticker Sheet - Red - OS,1,Sticker,Red,3.120000000,8.000000000,Accessory,Non-Core,Adult,2022-07-06,R,OS,Specialized Stickers
2024-06-06 00:00:00+00:00,2024-06-06,GBSTCK01,"GB Round Sticker 3"" x 3"" - RED - 3""",1,Sticker,RED,0.300000000,1.500000000,Accessory,Non-Core,All,2018-05-01,A,"3""",GB Edition
2024-06-06 00:00:00+00:00,2024-06-06,GBR1985YM,GB Edition S/S Youth Ranked Rashguard - Grey - YM,2,Rash Guard,Grey,9.850000000,42.990000000,Training,Core,Youth,2024-03-15,N,YM,Novo Estrada Limited
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-01 00:00:00+00:00,2022-09-01,GBJ2024WM,Stacked Puffer Jacket Womens - Mauve - M,1,Zip Up,Mauve,41.390000000,109.990000000,Casual,Non-Core,Adult,2020-11-25,D,M,Novo
2022-09-01 00:00:00+00:00,2022-09-01,GBK1809Y6,GBK Lite Ripstop Kimono w/ Belt - Blue - Y6,1,Kids Kimono,Blue,23.650000000,99.990000000,Training,Core,Youth,2018-09-13,A,Y6,Novo Estrada Limited
2022-09-01 00:00:00+00:00,2022-09-01,GBP19302XL,GB Legacy Lounge Pant - Black - 2XL,1,Lifestyle Pants,Black,15.000000000,49.990000000,Casual,Non-Core,Adult,2021-07-27,D,2XL,Novo Estrada Limited


If data already filled, do not run below code.

In [26]:
df

,ds,sku,line_item_name,quantity,product_type,product_color,cost,price_per_unit,category,sub_category,age,launch_date,status,variant_title,vendor
datetime,,,,,,,,,,,,,,,
2024-06-06 00:00:00+00:00,2024-06-06,GBT2228MXL,Legacy Mens Tee - Royal - XL,1,Shirt,Royal,7.310000000,29.990000000,Casual,Non-Core,Adult,2024-03-12,L,XL,Inkwright
2024-06-06 00:00:00+00:00,2024-06-06,GBT2204MXL,RS Mens L/S Tee - Black - XL,1,Shirt,Black,11.010000000,34.990000000,Casual,Non-Core,Adult,2023-12-15,R,XL,Inkwright
2024-06-06 00:00:00+00:00,2024-06-06,GBSTCK04,GB Assorted Matte Sticker Sheet - Red - OS,1,Sticker,Red,3.120000000,8.000000000,Accessory,Non-Core,Adult,2022-07-06,R,OS,Specialized Stickers
2024-06-06 00:00:00+00:00,2024-06-06,GBSTCK01,"GB Round Sticker 3"" x 3"" - RED - 3""",1,Sticker,RED,0.300000000,1.500000000,Accessory,Non-Core,All,2018-05-01,A,"3""",GB Edition
2024-06-06 00:00:00+00:00,2024-06-06,GBR1985YM,GB Edition S/S Youth Ranked Rashguard - Grey - YM,2,Rash Guard,Grey,9.850000000,42.990000000,Training,Core,Youth,2024-03-15,N,YM,Novo Estrada Limited
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-01 00:00:00+00:00,2022-09-01,GBJ2024WM,Stacked Puffer Jacket Womens - Mauve - M,1,Zip Up,Mauve,41.390000000,109.990000000,Casual,Non-Core,Adult,2020-11-25,D,M,Novo
2022-09-01 00:00:00+00:00,2022-09-01,GBK1809Y6,GBK Lite Ripstop Kimono w/ Belt - Blue - Y6,1,Kids Kimono,Blue,23.650000000,99.990000000,Training,Core,Youth,2018-09-13,A,Y6,Novo Estrada Limited
2022-09-01 00:00:00+00:00,2022-09-01,GBP19302XL,GB Legacy Lounge Pant - Black - 2XL,1,Lifestyle Pants,Black,15.000000000,49.990000000,Casual,Non-Core,Adult,2021-07-27,D,2XL,Novo Estrada Limited


In [27]:
df['launch_date'] = pd.to_datetime(df['launch_date']).dt.tz_localize(None)


# Fill missing categorical values and calculate product_age_days
columns_to_fill = ['launch_date', 'category', 'sub_category', 'product_type', 'age', 'vendor', 'variant_title', 'status', 'product_color']
for column in columns_to_fill:
    df[column] = df.groupby('sku')[column].ffill().bfill()

df['product_age_days'] = (df['ds'] - df['launch_date']).dt.days
df = df[df['ds'] >= df['launch_date']]

# Save and verify the prepared DataFrame
df.to_csv('prophet_fulfilled.csv', index=False)


C:\Users\erich\AppData\Local\Temp\ipykernel_21008\4034066324.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['launch_date'] = pd.to_datetime(df['launch_date']).dt.tz_localize(None)
C:\Users\erich\AppData\Local\Temp\ipykernel_21008\4034066324.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df.groupby('sku')[column].ffill().bfill()
C:\Users\erich\AppData\Local\Temp\ipykernel_21008\4034066324.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

Sku / cat data Model 2

## Sku-Wise Data Application

In [28]:
print(df.columns)


Index(['ds', 'sku', 'line_item_name', 'quantity', 'product_type',
       'product_color', 'cost', 'price_per_unit', 'category', 'sub_category',
       'age', 'launch_date', 'status', 'variant_title', 'vendor',
       'product_age_days'],
      dtype='object')


In [29]:
end_date = df['ds'].max()
start_date = end_date - pd.DateOffset(months=36)

# Function to calculate sales for specific 30 days periods
def calculate_period_sales(df, start_date, end_date):
    return df[(df['ds'] > start_date) & (df['ds'] <= end_date)]['quantity'].sum()

# Function to calculate the sales for 30D1, 30D2, and 30D3 periods
def calculate_sales_periods(df, last_sale_date):
    period_ends = last_sale_date
    period_starts = period_ends - timedelta(days=30)
    
    sales_30D1 = calculate_period_sales(df, period_starts, period_ends)
    
    period_ends = period_starts
    period_starts = period_ends - timedelta(days=30)
    sales_30D2 = calculate_period_sales(df, period_starts, period_ends)
    
    period_ends = period_starts
    period_starts = period_ends - timedelta(days=30)
    sales_30D3 = calculate_period_sales(df, period_starts, period_ends)
    
    return sales_30D1, sales_30D2, sales_30D3

# Modified weighted average calculation based on the new definition of periods
def weighted_avg_sales(sales_30D1, sales_30D2, sales_30D3):
    weighted_avg = (sales_30D1 * 0.5) + (sales_30D2 * 0.33) + (sales_30D3 * 0.17)
    return weighted_avg

# Aggregate information for each SKU, considering the updated periods
def aggregate_sku_data(group):
    last_sale_date = group['ds'].max()
    sales_30D1, sales_30D2, sales_30D3 = calculate_sales_periods(group, last_sale_date)
    avg_30x = weighted_avg_sales(sales_30D1, sales_30D2, sales_30D3)
    
    return pd.Series({
        '30D1': sales_30D1,
        '30D2': sales_30D2,
        '30D3': sales_30D3,
        '30X': avg_30x,
        'Total Sales': group['quantity'].sum(),
        'Last Sale Date': last_sale_date,
        'Launch Date': group['launch_date'].iloc[0],
        'Category': group['category'].iloc[0],
        'Sub_category': group['sub_category'].iloc[0],
        'Product_type': group['product_type'].iloc[0],
        'Age': group['age'].iloc[0],
        'Status': group['status'].iloc[0],
        'Product_color': group['product_color'].iloc[0],  
        'Cost': group['cost'].iloc[0],                  
        'Price_per_unit': group['price_per_unit'].iloc[0],  
        'Variant_title': group['variant_title'].iloc[0],    
        'Vendor': group['vendor'].iloc[0]              # Assuming you want the first entry
    })

# Use groupby on 'sku' and apply the aggregation function
skus_data = df.groupby('sku').apply(aggregate_sku_data).reset_index()

C:\Users\erich\AppData\Local\Temp\ipykernel_21008\1245201195.py:57: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  skus_data = df.groupby('sku').apply(aggregate_sku_data).reset_index()


In [ ]:
skus_data

In [31]:
skus_data.to_csv("skuseries.csv")

In [32]:
import pandas as pd
from datetime import timedelta

# Assuming 'df' is your DataFrame
end_date = df['ds'].max()
start_date = end_date - pd.DateOffset(months=36)

def calculate_period_sales(df, start_date, end_date):
    """ Calculate sales for a specified 30 days period. """
    return df[(df['ds'] > start_date) & (df['ds'] <= end_date)]['quantity'].sum()

def calculate_sales_periods(df, last_sale_date):
    """ Calculate the sales for the 30D1, 30D2, and 30D3 periods. """
    period_ends = last_sale_date
    period_starts = period_ends - timedelta(days=30)
    
    sales_30D1 = calculate_period_sales(df, period_starts, period_ends)
    period_ends = period_starts
    period_starts = period_ends - timedelta(days=30)
    sales_30D2 = calculate_period_sales(df, period_starts, period_ends)
    
    period_ends = period_starts
    period_starts = period_ends - timedelta(days=30)
    sales_30D3 = calculate_period_sales(df, period_starts, period_ends)
    
    return sales_30D1, sales_30D2, sales_30D3

def weighted_avg_sales(sales_30D1, sales_30D2, sales_30D3):
    """ Calculate the weighted average of sales. """
    return (sales_30D1 * 0.5) + (sales_30D2 * 0.33) + (sales_30D3 * 0.17)

def aggregate_sku_data(group):
    """ Aggregate data for each SKU. """
    last_sale_date = group['ds'].max()
    sales_30D1, sales_30D2, sales_30D3 = calculate_sales_periods(group, last_sale_date)
    avg_30x = weighted_avg_sales(sales_30D1, sales_30D2, sales_30D3)
    
    return pd.Series({
        '30D1': sales_30D1,
        '30D2': sales_30D2,
        '30D3': sales_30D3,
        '30X': avg_30x,
        'Total Sales': group['quantity'].sum(),
        'Last Sale Date': last_sale_date,
        'Launch Date': group['launch_date'].iloc[0],
        'Category': group['category'].iloc[0],
        'Sub_category': group['sub_category'].iloc[0],
        'Product_type': group['product_type'].iloc[0],
        'Age': group['age'].iloc[0],
        'Status': group['status'].iloc[0],
        'Product_color': group['product_color'].iloc[0],
        'Cost': group['cost'].iloc[0],
        'Price_per_unit': group['price_per_unit'].iloc[0],
        'Variant_title': group['variant_title'].iloc[0],
        'Vendor': group['vendor'].iloc[0]
    })

# Group by 'sku' and apply the aggregation function
skus_data = df.groupby('sku').apply(aggregate_sku_data).reset_index()

# Create a hybrid column 'Age_ProductType'
skus_data['Age_ProductType'] = skus_data['Age'].astype(str) + '-' + skus_data['Product_type']

# New categorical columns list
categorical_columns = ['Category', 'Sub_category', 'Age_ProductType', 'Product_color', 'Variant_title', 'Vendor']
metrics_columns = ['30D1', '30D2', '30D3', '30X', 'Total Sales']

# Group by the new set of categorical columns and sum the metrics
aggregated_metrics = skus_data.groupby(categorical_columns)[metrics_columns].sum().reset_index()

# Count unique SKUs for each group
sku_counts = skus_data.groupby(categorical_columns)['sku'].nunique().reset_index(name='SKU Count')

# Merge aggregated metrics and SKU counts
aggregated_data = pd.merge(aggregated_metrics, sku_counts, on=categorical_columns, how='left')

# Reset index
aggregated_data.reset_index(drop=True, inplace=True)

# Output to CSV
aggregated_data.to_csv("categories.csv")


C:\Users\erich\AppData\Local\Temp\ipykernel_21008\2632411032.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  skus_data = df.groupby('sku').apply(aggregate_sku_data).reset_index()


In [33]:
import pandas as pd

# Assuming 'skus_data' is your DataFrame
categorical_columns = ['Category', 'Sub_category', 'Product_type', 'Age', 'Status', 'Launch Date', 'Product_color', 'Variant_title']
metrics_columns = ['30D1', '30D2', '30D3', '30X', 'Total Sales']

# Group by all categorical columns and sum the metrics
aggregated_data = skus_data.groupby(categorical_columns)[metrics_columns].sum().reset_index()

# Calculate the count of unique SKUs for each combination
sku_counts = skus_data.groupby(categorical_columns)['sku'].nunique().reset_index(name='SKU Count')

# Merge the aggregated metrics with the SKU counts
aggregated_data = pd.merge(aggregated_data, sku_counts, on=categorical_columns, how='left')

# Create 'age_size' column by combining 'Age' and 'Variant_title' columns
aggregated_data['age_size'] = aggregated_data['Age'].astype(str) + '-' + aggregated_data['Variant_title'].astype(str)

# Pivot data to view one category against another
pivot_table = pd.pivot_table(aggregated_data, values='Total Sales', 
                             index=['Category', 'Product_color'], 
                             columns=['Product_type', 'age_size'],
                             aggfunc=sum, fill_value=0)

# Reset index if you want it in a flat format for easier export
pivot_table.reset_index(inplace=True)

# Export to CSV
pivot_table.to_csv("pivoter_nocolor_combined.csv", index=False)


C:\Users\erich\AppData\Local\Temp\ipykernel_21008\2061256183.py:20: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_table = pd.pivot_table(aggregated_data, values='Total Sales',


In [34]:
SCOPE = ["https://spreadsheets.google.com/feeds", 
         'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]
SERVICE_ACCOUNT_FILE = 'KEY_ID.json'  

credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, SCOPE)
client = gspread.authorize(credentials)

SPREADSHEET_ID = 'googlesheetsurl'
CSV_TO_SHEET_MAP = {
    "skuseries.csv": ("skucore", "A1"),
    "inventory.csv": ("inv", "A1"),
    "categories.csv": ("c_core", "A1"),
    "predictions.csv": ("pcast", "A1"),
    "pivoter_nocolor_combined.csv": ("gsr", "A1")
}

def upload_csv_to_sheet(csv_file, sheet_name, start_cell):
"""
If you're reading this, ask yourself. Why are we still using spreadsheets? Why are we still here?
"""
    df = pd.read_csv(csv_file)

    # Preprocess the DataFrame to replace NaN and infinities and truncate floats
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)  # Replace infinities and fill NaNs
    df = df.applymap(lambda x: round(x, 4) if isinstance(x, float) else x)  # Truncate floats

    # Access the Google Spreadsheet and the specific sheet
    spreadsheet = client.open_by_key(SPREADSHEET_ID)
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.WorksheetNotFound:
        # Create the sheet if it does not exist
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows="100", cols="20")
    
    # Clear existing data in the sheet to prevent data overlap
    worksheet.clear()
    
    # Convert DataFrame to a list of lists and include the header
    values = [df.columns.values.tolist()] + df.values.tolist()
    
    # Update the sheet with new data, ensuring the correct argument order
    worksheet.update(values, range_name=start_cell)

def main():
    # Iterate over the CSV files and their target Google Sheets and cells
    for csv_file, (sheet_name, start_cell) in CSV_TO_SHEET_MAP.items():
        if os.path.exists(csv_file):
            print(f"Uploading {csv_file} to {sheet_name} starting at {start_cell}...")
            upload_csv_to_sheet(csv_file, sheet_name, start_cell)
        else:
            print(f"File {csv_file} not found.")

if __name__ == "__main__":
    main()

IndentationError: expected an indented block after function definition on line 19 (1467025359.py, line 20)

In [ ]:
# Spreadsheet URL ID and desired CSVs with sheet and starting cell
# Product Performance Analysis
SPREADSHEET_ID = 'URL FOR ANALYSIS'
CSV_TO_SHEET_MAP = {
    "skuseries.csv": ("skucore", "A1"),
    "inventory.csv": ("inv", "A1"),
    "categories.csv": ("c_core", "A1"),
    "predictions.csv": ("pcast", "A1"),
    "pivoter_nocolors_synth.csv": ("gsr", "A1")
}
if __name__ == "__main__":
    main()